## Example of TRUE nanotribological properties workflow

### Background
- Using SAM as coating to reduce coefficient of friction and force of adhesion when shearing between two surfaces.
- Many variables that can be varied and, hence, makes it a prime candidate for Computational Simulation
- Several MD studies focuses on different properties
    - Andrew's Paper: backbone chain length, chain density, terminal groups
    - Current screening project: mixed composition 
    - Near future project: backbone chemitries
- The legacy encourage this research setup to be TRUE
    - Future research can continue this project by varing and/or adding more variables based on the original project
    
### Model
- Initialization of two surfaces coated with SAM - varying mixing composition, backbone chainlength, terminal groups
- Two surface was then energy minimized through a few steps (by LAMMPS and GROMACS)
- The two system is then compressed and sheared against each other (the top surface moving, and the bottom surface fixed)
- Output is analyzed with MDAnalysis

### Simulation Details
- Initialze with MoSDeF
- Fix overlaps with LAMMPS
- Energy Minimize with GROMACS
- NVT Equilibration with GROMACS
- Compress with GROMACS
- Shearing at 5nN with GROMACS
- Data Analysis with mdanalysis

#### System initialization

In [3]:
import mbuild
from util.helper.functions import system_builder
from util.helper.analysis import calc_nematic_order

In [ ]:
# Change into working directory
%cd example_simulation

In [ ]:
structure = system_builder(seed=1, chainlength=17, num_chains=50)

In [ ]:
structure.visualize()

#### Fix overlaps by LAMMPS

In [ ]:
# Command (lmp_mpi) might be different (lmp) depend on the installed lammps version
!lmp_mpi -in ../util/mdp_files/in.minimize -log minimize.log

#### Energy minimize by GROMACS

In [ ]:
# Convert .xtc to .gro
!echo 0| gmx trjconv -s init.gro -f minimize.xtc -o minimize.gro  -b 1.0 -e 1.0
# Grompp
!gmx grompp -f ../util/mdp_files/em.mdp -c minimize.gro -p init.top -n init.ndx -o em.tpr -maxwarn 2
# Run GROMACS
!gmx mdrun -v -deffnm em -s em.tpr -cpi em.cpt

#### NVT Equilibrate by GROMACS

In [ ]:
# Grompp
!gmx grompp -f ../util/mdp_files/nvt.mdp -c em.gro -p init.top -n init.ndx -o nvt.tpr -maxwarn 2
# Run GROMACS
!gmx mdrun -v -deffnm nvt -s nvt.tpr -cpi nvt.cpt

#### Compress wih GROMACS

In [ ]:
# Grompp
!gmx grompp -f ../util/mdp_files/compress.mdp -c nvt.gro -p init.top -n init.ndx -o compress.tpr -maxwarn 3
# Run GROMACS
!gmx mdrun -nt 16 -v -deffnm compress -s compress.tpr -cpi compress.cpt

#### Shearing at 5nN

In [ ]:
# Grompp
!gmx grompp -f ../util/mdp_files/shear_5nN.mdp -c compress.gro -p init.top -n init.ndx -o shear_5nN.tpr -maxwarn 3
# Run GROMACS
!gmx mdrun -v -nt 16 -s shear_5nN.tpr -deffnm shear_5nN -cpi shear_5nN.cpt -cpo shear_5nN.cpt -noappend

#### Calculate nematic order

In [ ]:
# Unwrap stuff

# Calculate nematic order
calc_nematic_order(traj_filename="shear_5nN-unwrapped.xtc", top_filename="shear_5nN.gro"
                   output_filename="shear_5n-S2.txt", ndx_filename="init.ndx", n_chains=50)

### Scaling up

- Above is a complete shearing simulation for a single system (a dual-monolayer with a 17 carbon Alkylsilane backbone cap with a methyl terminal group). The ouput result can be used to calculate wide range of tribology properties such as coefficient of friction, force of adhesion, or nematic order (as shown above). However, to study the trends of the se tribological properties induced by the terminal groups, backbone chemistries, or backbone chainlength, etc., we will need to have more than just a few simulations. Hence, design a project with TRUE in mind, will be extremely beneficial in the long run, when we desire to study more variables, extending beyond the original project.
- The following example will try to show how, by designing our code following the Object Oriented Principles (OOP) and using a few open-source libraries, we can build a TRUE simulation project, emphasizing at its scalability. 

In [ ]:
%cd ../example_workspace/

In [ ]:
!python src/init.py --seed 27 --num-replicas 3 

In [ ]:
!python src/project.py run -o initialize_system